# Tutorial

In [ ]:
pip install -U git+https://github.com/sky1ove/af_kit.git

## MSA

MSA can be run in cpu only server

In [ ]:
pip install "colabfold[alphafold] @ git+https://github.com/sokrypton/ColabFold"

Prepare a csv that have first column `id` and second column `sequence` of amino acid sequence

In [ ]:
colabfold_batch file.csv folder_name --msa-only

After finish, copy a3m files to a gpu available place

In [ ]:
subfolder='project_name'

In [ ]:
copy_a3m(a3m_dir=f'/teamspace/studios/alphfold3/msa_{subfolder}',
         dest_dir=f'af_input/{subfolder}/msa')

## JSON file

In [ ]:
from af_kit.core import *
import pandas as pd
from tqdm import tqdm

Read the file that contained id and sequence

In [ ]:
df = pd.read_csv('file.csv')

In [ ]:
for idx, row in tqdm(df.iterrows(),total=len(df)):
    data = get_AF_input([row.id], # if multiple protein, put their id in the []
                        a3m_dir=f'af_input/{subfolder}/msa',
                        save_folder=f'af_input/{subfolder}'
                        )

This will generate a number of json files in the save_folder. 

We need to distribute them to nfolders for parallel running when multiple gpus are available.

In [ ]:
split_nfolder(f'af_input/{subfolder}',n=4) # default n is 4

## Docker Command

In [ ]:
docker pull sky1ove/alphafold3

In [ ]:
for i in range(4):
    get_docker_command(input_dir=f"af_input/{subfolder}/folder_{i}",
                       output_dir=f"af_output/{subfolder}",
                       gpus=i)

Run the printed command in your terminal

## Report for protein pairs

In [ ]:
from af_kit.analyze import *

In [ ]:
df_sum, top_genes = get_report(f"af_output/{subfolder}",
                               save_dir=f'af_report/{subfolder}')

df_sum.sort_values('iptm_ptm_rnk_add').head(10)

A 3d plot will be generated with x='iptm',y='ptm',z='chain_pair_pae_min_add'

Top genes are:
- Smallest 30 from 'iptm_ptm_rnk_add', 
        'chain_pair_pae_min_add', 
        'chain_pair_pae_min_0_1', 
        'chain_pair_pae_min_1_0', 
        'iptm_pae_add_rnk'
- Largest 30 from 'ranking_score', 'iptm', 'iptm_ptm_add'

df_sum contains the score for each metric

## Embeddings

To do 